# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [10]:
#Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

In [3]:
#changing some columns data types
spaceship = spaceship.astype({"CryoSleep": bool, "VIP": bool})

#cabin column remapping
spaceship["Cabin"] = spaceship["Cabin"].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x) 

#dropping passengerid & name column
spaceship = spaceship.drop(["PassengerId", "Name"], axis = 1)

#dropping null values and checking after
df = spaceship.dropna(how="any")

#doing dummies for non-numerical
df = pd.get_dummies(df, columns=["HomePlanet", "Cabin", "Destination" ], dtype = int)
df

CryoSleep   Age    VIP  RoomService  FoodCourt  ShoppingMall     Spa  \
0         False  39.0  False          0.0        0.0           0.0     0.0   
1         False  24.0  False        109.0        9.0          25.0   549.0   
2         False  58.0   True         43.0     3576.0           0.0  6715.0   
3         False  33.0  False          0.0     1283.0         371.0  3329.0   
4         False  16.0  False        303.0       70.0         151.0   565.0   
...         ...   ...    ...          ...        ...           ...     ...   
8688      False  41.0   True          0.0     6819.0           0.0  1643.0   
8689       True  18.0  False          0.0        0.0           0.0     0.0   
8690      False  26.0  False          0.0        0.0        1872.0     1.0   
8691      False  32.0  False          0.0     1049.0           0.0   353.0   
8692      False  44.0  False        126.0     4688.0           0.0     0.0   

      VRDeck  Transported  HomePlanet_Earth  ...  Cabin_B  Cabin_C  Cabin_D  \
0        0.0        False                 0  ...        1        0        0   
1       44.0         True                 1  ...        0        0        0   
2       49.0        False                 0  ...        0        0        0   
3      193.0        False                 0  ...        0        0        0   
4        2.0         True                 1  ...        0        0        0   
...      ...          ...               ...  ...      ...      ...      ...   
8688    74.0        False                 0  ...        0        0        0   
8689     0.0        False                 1  ...        0        0        0   
8690     0.0         True                 1  ...        0        0        0   
8691  3235.0        False                 0  ...        0        0        0   
8692    12.0         True                 0  ...        0        0        0   

      Cabin_E  Cabin_F  Cabin_G  Cabin_T  Destination_55 Cancri e  \
0           0        0        0        0                        0   
1           0        1        0        0                        0   
2           0        0        0        0                        0   
3           0        0        0        0                        0   
4           0        1        0        0                        0   
...       ...      ...      ...      ...                      ...   
8688        0        0        0        0                        1   
8689        0        0        1        0                        0   
8690        0        0        1        0                        0   
8691        1        0        0        0                        1   
8692        1        0        0        0                        0   

      Destination_PSO J318.5-22  Destination_TRAPPIST-1e  
0                             0                        1  
1                             0                        1  
2                             0                        1  
3                             0                        1  
4                             0                        1  
...                         ...                      ...  
8688                          0                        0  
8689                          1                        0  
8690                          0                        1  
8691                          0                        0  
8692                          0                        1  

[7111 rows x 23 columns]

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [19]:
df["target"] = df["Transported"]

features = df.drop(columns=["Transported"])
target = df["Transported"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [20]:
normalizer = MinMaxScaler()
normalizer.fit(X_train)

MinMaxScaler()

In [21]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

knn = KNeighborsRegressor(n_neighbors = 10)
knn.fit(X_train_norm, y_train)

KNeighborsRegressor(n_neighbors=10)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [24]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())
ada_reg.fit(X_train_norm, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor())

- Evaluate your model

In [25]:
pred = ada_reg.predict(X_test_norm)

print("R2 score", ada_reg.score(X_test_norm, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("MAE", mean_absolute_error(pred, y_test))

R2 score 1.0
RMSE 0.0
MAE 0.0


C:\Users\Amira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [26]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [27]:
model = GridSearchCV(estimator = ada_reg, param_grid = grid, cv=5)

- Run Grid Search

In [28]:
model.fit(X_train_norm, y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor()),
             param_grid={'estimator__max_depth': [10, 30, 50],
                         'estimator__max_leaf_nodes': [250, 500, 1000, None],
                         'n_estimators': [50, 100, 200, 500]})

- Evaluate your model

In [29]:
model.best_params_

{'estimator__max_depth': 10,
 'estimator__max_leaf_nodes': 250,
 'n_estimators': 50}

In [30]:
best_model = model.best_estimator_

In [31]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

MAE 0.0
RMSE 0.0
R2 score 1.0


C:\Users\Amira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
